In [1]:
import datetime as dt
import time
import re


class Utils:
    
    def __init__(self):
        pass
    
    @staticmethod
    def date_formats(date):
        # Comprobamos el formato de la fecha string de entrada
#         pattern = re.compile(r'\d{4}\-[0-1]\d\-[0-3]\d\s[0-2]\d\:[0-5]\d\:[0-5]\d') # Mejorable, quizas mejor hacerlo en la conversion a datetime
        date_string = date
        date_list = re.split('-| |:', date)
        date_datetime = dt.datetime(*[int(x) for x in date_list], tzinfo=dt.timezone.utc) # Mirar meter un try/except
        date_unix = int(time.mktime(date_datetime.timetuple()))
        formats = {'string': date_string, 'list': date_list, 'datetime': date_datetime, 'unix': date_unix}
        return(formats)

In [2]:
import krakenex
from pykrakenapi import KrakenAPI
# from ______ import Utils
import numpy as np
import time


class Downloader:
    
    def __init__(self, API=KrakenAPI(krakenex.API())):
        self.API = API
        self.data = {}
    
    def download_split_historic(self, pair, dt_from):
        # Obtenemos los 1000 registro desde la fecha indicada
        trades, last = self.API.get_recent_trades(pair, dt_from)
        # Ordenamos por fecha
        trades = trades.sort_values('time', ascending=False)
        trades = trades.reset_index()
        # Obtenemos primero y ultimo
        first = trades['time'].iloc[-1]
        last = trades['time'].iloc[0]
        return({'first': first, 'last': last, 'trades': trades})
        
    def download_fromto_historic(self, pair, dt_from, dt_to):
        data_pair = []
        # Inicializamos while para descargar fromto historico
        from_dynamic = dt_from
        while from_dynamic<=dt_to:
            split = self.download_split_historic(pair, from_dynamic)
            data_pair.append(split)
            from_dynamic = split['last']
            print('Descargado de '+str(split['trades']['dtime'].iloc[-1])+' a '+str(split['trades']['dtime'].iloc[0]))
            # Dormimos la descarga
            time.sleep(np.random.uniform(1,2))
        # Guardamos el acumulado en el atributo de clase data
        self.data[pair] = data_pair

In [3]:
import pandas as pd


class Wrangler:
    
    def __init__(self):
        pass
    
    def get_clean_data(self, raw_data):
        clean_data = {}
        # Iteramos sobre los pares en raw_data y consolidamos un dataframe
        for k, v in raw_data.items():
            pair_data = pd.concat([x['trades'] for x in v])
            pair_data = pair_data.drop_duplicates()
            pair_data = pair_data[['dtime', 'time', 'price', 'volume']]
            pair_data = pair_data.sort_values('time', ascending=False)
            clean_data[k] = pair_data
        return(clean_data)

In [4]:
class Calculator:
    
    def __init__(self, data):
        self.data = {}
        self.data['original'] = data

    def compute_price(self, price='price', volume='volume', time=['dtime', 'time']):
        price_data = self.data['original'].copy()
        # Calculamos precio por volumen
        price_data[price+'_x_'+volume] = price_data[price]*price_data[volume]
        # Agregamos
        pxv = price+'_x_'+volume
        d = {volume: np.sum, pxv: np.sum}
        price_data = price_data.groupby(time).agg(d)
        # Recalculamos el precio
        price_data[price] = price_data[pxv]/price_data[volume]
        # Seleccionamos las variables
        price_data = price_data[[price, volume]].reset_index()
        # Guardamos
        self.data['price'] = price_data       
        
    def compute_vwap(self, price='price', volume='volume', time=['dtime', 'time'], minutes_interval=15):
        vwap_data = self.data['original'].copy()
        # Generamos los bins 
        seconds = minutes_interval*60
        min_period = int(np.floor(min(vwap_data['time']/seconds))*seconds-seconds)
        max_period = int(np.ceil(max(vwap_data['time']/seconds))*seconds+seconds)
        bins = list(range(min_period, max_period, seconds))
        vwap_data['new_time'] = pd.cut(vwap_data[time[1]], bins=bins)
        # Calculamos precio por volumen
        vwap_data[price+'_x_'+volume] = vwap_data[price]*vwap_data[volume]
        # Agregamos
        pxv = price+'_x_'+volume
        d = {volume: np.sum, pxv: np.sum}
        vwap_data = vwap_data.groupby('new_time').agg(d)
        # Recalculamos el precio
        vwap_data[price] = vwap_data[pxv]/vwap_data[volume]
        # Reteamos indice y eliminamos nas
        vwap_data = vwap_data[[price, volume]].reset_index()
        # Formateamos la fecha
        vwap_data['time'] = [int(x[-11:-1]) for x in vwap_data['new_time'].astype(str)]
        vwap_data['dtime'] = [dt.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in vwap_data['time']]
        # Aseguramos el orden por fecha 
        vwap_data = vwap_data.sort_values('time', ascending=True)
        # Rellenamos los precios vacios
        vwap_data['price'] = vwap_data['price'].fillna(method='ffill')
        vwap_data['price'] = vwap_data['price'].fillna(method='bfill')
        # Ordenamos variables
        vwap_data = vwap_data[time+[price, volume]]
        # Guardamos
        self.data['vwap'] = vwap_data

In [5]:
# dt_from = '2020-11-01 01:25:17'
# Utils.date_formats(dt_from)

In [6]:
pair = "BCHUSD"
dt_from = Utils.date_formats('2021-10-01 00:00:00')
dt_to = Utils.date_formats('2021-10-02 00:00:00')
# Generamos los formatos para las fechas 
d = Downloader()
# aux = d.download_split_historic(pair, dt_from['unix'])
d.download_fromto_historic(pair, dt_from['unix'], dt_to['unix'])

Descargado de 2021-09-30 22:01:56.311099904 a 2021-10-01 10:29:18.337799936
Descargado de 2021-10-01 10:29:18.098700032 a 2021-10-01 12:46:41.971399936
Descargado de 2021-10-01 12:46:41.971399936 a 2021-10-02 14:47:49.193200128


In [7]:
w = Wrangler()
trades = w.get_clean_data(d.data)

In [8]:
c = Calculator(trades[pair])
c.compute_price()
c.compute_vwap()

In [9]:
c.data

{'original':                             dtime          time   price    volume
 0   2021-10-02 14:47:49.193200128  1.633186e+09  549.79  0.274112
 1   2021-10-02 14:47:49.176100096  1.633186e+09  549.79  0.274131
 2   2021-10-02 14:45:29.279800064  1.633186e+09  549.78  5.000000
 3   2021-10-02 14:41:05.274899968  1.633186e+09  549.37  0.378434
 4   2021-10-02 14:39:25.826600192  1.633186e+09  549.14  0.063505
 ..                            ...           ...     ...       ...
 995 2021-09-30 22:07:30.520800000  1.633040e+09  500.85  2.424802
 996 2021-09-30 22:04:41.415200000  1.633039e+09  500.85  0.855198
 997 2021-09-30 22:04:14.451899904  1.633039e+09  500.86  1.000000
 998 2021-09-30 22:02:20.607000064  1.633039e+09  500.69  0.599000
 999 2021-09-30 22:01:56.311099904  1.633039e+09  501.30  2.390000
 
 [2996 rows x 4 columns],
 'price':                              dtime          time   price    volume
 0    2021-09-30 22:01:56.311099904  1.633039e+09  501.30  2.390000
 1    2021-